# Costa Rican Household Poverty Level Prediction - XGBoost

참고 : https://www.kaggle.com/skooch/xgboost

# 1st

# LGMB with random split for early stopping

- 이 커널은 (가구의 집계를 추출한 후) 가정의 가장만 학습시킨다. 점수는 가정의 가장으로만 평가된다. 모든 가족구성원은 test + sample submission에 포함되지만, 가정만 점수가 매겨진다. 하지만, 현재 평가는 가장이 아닌 가족구성원에게도 이루어지는 것으로 보인다.
- 클래스 빈도의 균형을 맞추는 것은 아주 중요하다. 균형을 맞추지 않으면 학습된 모델은 성능이 낮다. 이 작업은 직접 할 수도 있고, 언더샘플링을 할 수도 있다. 하지만 가장 간단한 것은 (그리고 언더샘플링보다 더 강력한 것은) sklearn API의 LightGBM모델 생성시 class_weight='balanced'로 설정하는 것이다.
- 이 커널은 macro F1 score를 사용해 학습을 조기중단한다. 이 작업은 scoring 전략에 맞춰 수행한다.
- 범주형 변수는 blind label encoding 대신 적절히 매핑된 숫자로 변환된다.
- OHE(One-Hot Encoding)가 label encoding되면, 트리 모델에서 digest하기 더 쉽다. 이 트릭은 트리모델이 아닌 모델에 안 좋을 수 있으니 조심해야한다.
- idhogar은 학습에 사용되지 않는다. 정보를 얻는 유일한 방법은 데이터 누락이 없는 경우다. 
- 가정 내에서 집계가 이루어지며 새로운 변수를 직접 생성한다. 대부분의 변수는 가정 수준에서 이미 사용되기 때문에 집계될 수 있는 변수가 많지 않다.
- voting classifier은 여러 LgithGBM 모델의 평균을 내는 데 사용한다.

In [5]:
import numpy as np   # linear algebra
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import f1_score
#from sklearn.externals.joblib import Parallel, delayed
from joblib import Parallel, delayed
from sklearn.base import clone
from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.utils import class_weight

import warnings
warnings.filterwarnings('ignore')

범주형 변수 매핑

In [6]:
from sklearn.preprocessing import LabelEncoder

# 여기선 idhogar 필드만 변환. 이 함수는 다른 데에서도 사용
def encode_data(df):
    df['idhogar'] = LabelEncoder().fit_transform(df['idhogar'])
    
# sklearn의 의사결정나무를 위한 변수 중요도 plot
def feature_importance(forest, X_train, display_results=True):
    ranked_list = []
    zero_features = []
    
    importances = forest.freature_importances_
    indices = np.argsort(importances)[::-1]
    
    if display_results:
        # 변수 순위 출력
        print('Feature ranking :')
    
    for f in range(X_train.shape[1]):
        if display_results:
            print('%d. feature %d (%f)' %(f+1, indices[f], importances[indices[f]])
                  + ' - ' + X_train.columns(indicies[f]))
        ranked_list.append(X_train.columns[indices[f]])
        
        if importances[indices[f]] == 0.0:
            zero_features.append(X_train.columns[indices[f]])
    
    return ranked_list, zero_features

In [8]:
def do_features(df):
    feats_div = [('children_fraction', 'r4t1', 'r4t3'), 
                 ('working_man_fraction', 'r4h2', 'r4t3'),
                 ('all_man_fraction', 'r4h3', 'r4t3'),
                 ('human_density', 'tamviv', 'rooms'),
                 ('human_bed_density', 'tamviv', 'bedrooms'),
                 ('rent_per_person', 'v2a1', 'r4t3'),
                 ('rent_per_room', 'v2a1', 'rooms'),
                 ('mobile_density', 'qmobilephone', 'r4t3'),
                 ('tablet_density', 'v18q1', 'r4t3'),
                 ('mobile_adult_density', 'qmobilephone', 'r4t2'),
                 ('tablet_adult_density', 'v18q1', 'r4t2')]
    feats_sub = [('people_not_living', 'tamhog', 'tamviv'),
                 ('people_weird_stat', 'tamhog', 'r4t3')]
    
    for f_new, f1, f2 in feats_div:
        df['fe_' + f_new] = (df[f1] / df[f2]).astype(np.float32)
    for f_new, f1, f2 in feats_sub:
        df['fe_' + f_new] = (df[f1] - df[f2]).astype(np.float32)
        
    # 가정으로 집계 규칙
    aggs_num = {'age': ['min','max','mean'], 'escolari': ['min','max','mean']}
    aggs_cat = {'dis':['mean']}
    for s_ in ['estadocivil','parentesco','instlevel']:
        for f_ in [f_ for f_ in df.columns if f_.startswith(s_)]:
            aggs_cat[f_] = ['mean','count']
            
    # 가장으로 집계
    for name_, df_ in [('18', df.query('age>=18'))]:
        df_agg = df_.groupby('idhogar').agg({**aggs_num, **aggs_cat}).astype(np.float32)
        df_agg.columns = pd.Index(['agg' + name_ + '_' + e[0] + '_'
                                  + e[1].upper() for e in df_agg.columns.tolist()])
        df = df.join(df_agg, how='left', on='idhogar')
        del df_agg
        
    # id 삭제
    df.drop(['Id'], axis=1, inplace=True)
    
    return df

In [9]:
# one hot encoding된 필드를 laebl encoding으로 변경
def convert_OHE2LE(df):
    tmp_df = df.copy(deep=True)
    for s_ in ['pared', 'piso', 'techo', 'abastagua', 'sanitario', 'energcocinar', 'elimbasu', 
               'epared', 'etecho', 'eviv', 'estadocivil', 'parentesco', 'instlevel', 'lugar', 'tipovivi',
               'manual_elec']:
        if 'manual_' not in s_:
            cols_s_ = [f for f_ in df.columns if f_.startwith(s)]  
        elif 'elec' in s_:
            cols_s_ = ['public','planpri','noelec','coopele']
        sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()
        
        # columns==0인 합계가 있는 OHE 열을 처리
        if 0 in sum_ohe:
            print('The OHE in {} is incomplete. A new column will be added before label encoding'.format(s_))
            # 더미변수 이름 추가
            col_dummy = s_ + '_dummy'
            # 데이터프레임에 열 추가
            tmp_df[col_dummy] = (tmp_df[cols_s_].sum(axis=1)==0).astype(np.int8)
            # label encoding 위해 열 리스트에 이름 추가
            cols_s_.append(col_dummy)
            # 범주 인코딩이 완료된 것을 증명
            sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()
            if 0 in sum_ohe:
                print('The category completion did not work')
                
        tmp_cat = tmp_df[cols_s_].idxmax(axis=1)
        tmp_df[s_ + '_LS'] = LabelEncoder().fit_transfor(tmp_cat).astype(np.int16)
        if 'parentesco1' in cols_s_:
            cols_s_.remove('parentesco1')
        tmp_df.drop(cols_s_, axis=1, inplace=True)
    
    return tmp_df